In [5]:
from random import choice, randint, random
from string import ascii_lowercase
from copy import deepcopy

#https://github.com/100/Solid/edit/master/Solid/TabuSearch.py
from abc import ABCMeta, abstractmethod
from copy import deepcopy
from collections import deque
from numpy import argmax

class TabuSearch:
    """
    Conducts tabu search
    """
    __metaclass__ = ABCMeta

    cur_steps = None

    tabu_size = None
    tabu_list = None

    initial_state = None
    current = None
    best = None

    max_steps = None
    max_score = None

    def __init__(self, initial_state, tabu_size, max_steps, max_score=None):
        """

        :param initial_state: initial state, should implement __eq__ or __cmp__
        :param tabu_size: number of states to keep in tabu list
        :param max_steps: maximum number of steps to run algorithm for
        :param max_score: score to stop algorithm once reached
        """
        self.initial_state = initial_state

        if isinstance(tabu_size, int) and tabu_size > 0:
            self.tabu_size = tabu_size
        else:
            raise TypeError('Tabu size must be a positive integer')

        if isinstance(max_steps, int) and max_steps > 0:
            self.max_steps = max_steps
        else:
            raise TypeError('Maximum steps must be a positive integer')

        if max_score is not None:
            if isinstance(max_score, (int, float)):
                self.max_score = float(max_score)
            else:
                raise TypeError('Maximum score must be a numeric type')

    def __str__(self):
        return ('TABU SEARCH: \n' +
                'CURRENT STEPS: %d \n' +
                'BEST SCORE: %f \n' +
                'BEST MEMBER: %s \n\n') % \
               (self.cur_steps, self._score(self.best), str(self.best))

    def __repr__(self):
        return self.__str__()

    def _clear(self):
        """
        Resets the variables that are altered on a per-run basis of the algorithm

        :return: None
        """
        self.cur_steps = 0
        self.tabu_list = deque(maxlen=self.tabu_size)
        self.current = self.initial_state
        self.best = self.initial_state

    @abstractmethod
    def _score(self, state):
        """
        Returns objective function value of a state

        :param state: a state
        :return: objective function value of state
        """
        pass

    @abstractmethod
    def _neighborhood(self):
        """
        Returns list of all members of neighborhood of 
        state, given self.current

        :return: list of members of neighborhood
        """
        pass

    def _best(self, neighborhood):
        """
        Finds the best member of a neighborhood

        :param neighborhood: a neighborhood
        :return: best member of neighborhood
        """
        return neighborhood[argmax([self._score(x) for x in neighborhood])]

    def run(self, verbose=True):
        """
        Conducts tabu search

        :param verbose: indicates whether or not to print progress regularly
        :return: best state and objective function value of best state
        """
        self._clear()
        for i in range(self.max_steps):
            self.cur_steps += 1

            if ((i + 1) % 100 == 0) and verbose:
                print(self)

            neighborhood = self._neighborhood()
            neighborhood_best = self._best(neighborhood)

            while True:
                if all([x in self.tabu_list for x in neighborhood]):
                    print("TERMINATING - NO SUITABLE NEIGHBORS")
                    return self.best, self._score(self.best)
                if neighborhood_best in self.tabu_list:
                    if self._score(neighborhood_best) > self._score(self.best):
                        self.tabu_list.append(neighborhood_best)
                        self.best = deepcopy(neighborhood_best)
                        break
                    else:
                        neighborhood.remove(neighborhood_best)
                        neighborhood_best = self._best(neighborhood)
                else:
                    self.tabu_list.append(neighborhood_best)
                    self.current = neighborhood_best
                    if self._score(self.current) > self._score(self.best):
                        self.best = deepcopy(self.current)
                    break

            if self.max_score is not None and self._score(self.best) > self.max_score:
                print("TERMINATING - REACHED MAXIMUM SCORE")
                return self.best, self._score(self.best)
        print("TERMINATING - REACHED MAXIMUM STEPS")
        return self.best, self._score(self.best)


In [90]:
class Algorithm(TabuSearch):
    """
    Tries to get a randomly-generated string to match string "clout"
    """
    def _neighborhood(self):
        member = list(self.current)
        neighborhood = []
        for _ in range(10):
            neighbor = deepcopy(member)
            neighbor[randint(0, 44)] = choice(ascii_lowercase)
            neighbor = ''.join(neighbor)
            neighborhood.append(neighbor)
        return neighborhood

    def _score(self, state):
        return float(sum(state[i] == "123456789012345678901234567890123456789012345"[i] for i in range(45)))


def test_algorithm():
    algorithm = Algorithm('abcdefghijabcdefghijabcdefghijabcdefghijabcde', 50, 500, max_score=None)
    algorithm.run()

In [145]:
class Algorithm(TabuSearch):
    """
    Tries to get a randomly-generated string to match string "clout"
    """
    def _neighborhood(self):
        member = list(self.current)
        neighborhood = []
        for _ in range(10):
            neighbor = deepcopy(member)
            neighbor[randint(0, 9)] = choice(ascii_lowercase)
            #neighbor = ''.join(neighbor) #list to string
            neighborhood.append(neighbor)
        return neighborhood

    def _score(self, state):
        return float(sum(state[i] == ['a','a','a','a','a','a','a','a','a','a'][i] for i in range(10)))


def test_algorithm():
    algorithm = Algorithm(['b','b','b','b','b','b','b','b','b','b'], 50, 500, max_score=None)
    algorithm.run()

In [147]:
 test_algorithm()

TABU SEARCH: 
CURRENT STEPS: 100 
BEST SCORE: 9.000000 
BEST MEMBER: ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'f', 'a', 'a'] 


TABU SEARCH: 
CURRENT STEPS: 200 
BEST SCORE: 9.000000 
BEST MEMBER: ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'f', 'a', 'a'] 


TABU SEARCH: 
CURRENT STEPS: 300 
BEST SCORE: 10.000000 
BEST MEMBER: ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'] 


TABU SEARCH: 
CURRENT STEPS: 400 
BEST SCORE: 10.000000 
BEST MEMBER: ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'] 


TABU SEARCH: 
CURRENT STEPS: 500 
BEST SCORE: 10.000000 
BEST MEMBER: ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'] 


TERMINATING - REACHED MAXIMUM STEPS
